In [1]:
import json

import sys
sys.version

'3.9.9 (main, Dec 21 2021, 10:14:07) \n[GCC 10.2.1 20210110]'

In [2]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

from datetime import datetime
from datetime import timezone, timedelta

# UTC to JST
def change_timeslot(timeslot):
    timeslot_dict = []
    for oneslot in timeslot:
        _utc = re.split('[T.]', oneslot)
        utc = _utc[0] + " " + _utc[1]
        utc_time = datetime.strptime(utc, '%Y-%m-%d %H:%M:%S')
        _jst_time = utc_time.astimezone(timezone(timedelta(hours=+9)))
        jst_time = datetime.strftime(_jst_time, '%Y-%m-%d %H:%M:%S')

        timeslot_dict.append({'utc_time':oneslot, 'local_time':jst_time})
    return timeslot_dict

class ElasticAPI():
    def __init__(self, es_server):
        self.es = Elasticsearch(es_server)

    # check index and create alias
    def check_indices(self, index_name):
        es = self.es
        indices = es.cat.indices(index='*', h='index').splitlines()
        if index_name not in indices:
            es.indices.create(index=index_name)

            alias = 'search_uuid'
            es.indices.update_aliases(
                actions = [
                    { 'add': { 'index': index_name, 'alias': alias }}
                ]
            )

    # put the data from Prism(Element) API to Elasticsearch
    def put_rest_pe(self, r_json, timestamp, cluster_name, cluster_uuid, index_name):
        es = self.es
        index_name = index_name
        self.check_indices(index_name)

        actions = []
        if index_name != 'share_details':
            for entity in r_json['entities']:
                entity['timestamp'] = timestamp
                entity['cluster_name'] = cluster_name
                entity['cluster_uuid'] = cluster_uuid
                actions.append({'_index':index_name, '_source':entity})
        else:
            for entity in r_json:
                entity['timestamp'] = timestamp
                entity['cluster_name'] = cluster_name
                entity['cluster_uuid'] = cluster_uuid
                actions.append({'_index':index_name, '_source':entity})

        reaction = helpers.bulk(es, actions)
        return reaction[0]

    def input_data(self, res_list):
        timestamp = datetime.utcnow()
        input_size = {}

        # vms
        vms_json = res_list['vms']
        cluster_name = vms_json['entities'][0]['spec']['cluster_reference']['name']
        cluster_uuid = vms_json['entities'][0]['spec']['cluster_reference']['uuid']
        input_size['vms'] = self.put_rest_pe(vms_json, timestamp, cluster_name, cluster_uuid, index_name='vms')

        # volume_group
        volume_groups_json = res_list['volume_groups']
        input_size['volume_groups'] = self.put_rest_pe(volume_groups_json, timestamp, cluster_name, cluster_uuid, index_name='volume_groups')

        return cluster_name, input_size

In [3]:
ELASTIC_SERVER = 'http://elasticsearch:9200'
es = ElasticAPI(es_server=ELASTIC_SERVER)


In [4]:
res_list = {}
with open('./sample/vms.json') as f:
    res_list['vms'] = json.load(f)
with open('./sample/volume_groups.json') as f:
    res_list['volume_groups'] = json.load(f)

In [5]:
es.input_data(res_list)

/tmp/ipykernel_1860/2895218574.py:27: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  indices = es.cat.indices(index='*', h='index').splitlines()
/tmp/ipykernel_1860/2895218574.py:29: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  es.indices.create(index=index_name)
/tmp/ipykernel_1860/2895218574.py:32: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  e

('POC20', {'vms': 35, 'volume_groups': 32})